# 🎓 Chapter 2: Markov Decision Processes — Hands-On Lab

**Complete Reinforcement Learning Journey: From Basics to RLHF**

In this notebook, you will:
1. **Build 7 MDP environments** from scratch
2. **Visualize** states, actions, transitions, and rewards
3. **Understand** deterministic vs stochastic dynamics
4. **Compute** state-value V(s) and action-value Q(s,a)
5. **See** how γ (discount factor) changes optimal behavior

---

📘 **Companion to Chapter 2 of the book**  
🔗 Interactive web app: [MDP Explorer](https://mlnjsh.github.io/rl-book-labs/ch2/)  
🔗 GitHub: [github.com/mlnjsh/rl-book-labs](https://github.com/mlnjsh/rl-book-labs)

## 📦 Install & Import Libraries

In [ ]:
# Install required libraries (run once)
!pip install gymnasium numpy matplotlib seaborn pandas --quiet

# All libraries used in this notebook:
# ┌─────────────────────────────────────────────────────────┐
# │ gymnasium    - RL environments (FrozenLake, etc.)       │
# │ numpy        - numerical computation                    │
# │ matplotlib   - plotting and visualization               │
# │ seaborn      - heatmaps for value functions             │
# │ pandas       - tables for transition probabilities      │
# │ IPython      - display utilities (built-in)             │
# └─────────────────────────────────────────────────────────┘

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import pandas as pd
from IPython.display import display, HTML, Markdown
import gymnasium as gym
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Style
plt.rcParams['figure.facecolor'] = '#0f172a'
plt.rcParams['axes.facecolor'] = '#1e293b'
plt.rcParams['text.color'] = '#e2e8f0'
plt.rcParams['axes.labelcolor'] = '#e2e8f0'
plt.rcParams['xtick.color'] = '#94a3b8'
plt.rcParams['ytick.color'] = '#94a3b8'
plt.rcParams['font.family'] = 'monospace'

print("✅ All libraries loaded!")
print(f"   gymnasium: {gym.__version__}")
print(f"   numpy: {np.__version__}")

---
## 🏗️ MDP Framework

Every MDP is a tuple **M = ⟨S, A, P, R, γ⟩**:
- **S** — set of states
- **A** — set of actions
- **P(s'|s,a)** — transition probabilities
- **R(s,a,s')** — reward function
- **γ** — discount factor

Let's build a base class that all our environments will use:

In [ ]:
class MDP:
    """Base MDP class that all environments inherit from."""

    def __init__(self, states, actions, transitions, rewards, gamma=0.9):
        """
        states: list of state names
        actions: dict {state: [available_actions]}
        transitions: dict {(state, action): [(prob, next_state), ...]}
        rewards: dict {(state, action, next_state): reward}
        gamma: discount factor
        """
        self.states = states
        self.actions = actions
        self.transitions = transitions
        self.rewards = rewards
        self.gamma = gamma

    def get_transitions(self, state, action):
        """Returns list of (probability, next_state, reward) tuples."""
        results = []
        for prob, next_state in self.transitions.get((state, action), []):
            reward = self.rewards.get((state, action, next_state), 0)
            results.append((prob, next_state, reward))
        return results

    def print_summary(self):
        """Print the MDP components."""
        print(f"{'='*50}")
        print(f"  MDP Summary")
        print(f"{'='*50}")
        print(f"  States  (|S|={len(self.states)}): {self.states}")
        print(f"  Actions: {set(a for acts in self.actions.values() for a in acts)}")
        print(f"  Gamma:   {self.gamma}")
        print(f"  Transitions: {len(self.transitions)} (state,action) pairs")
        print(f"{'='*50}")

    def show_transitions_table(self, state, action):
        """Show transition table for a (state, action) pair."""
        ts = self.get_transitions(state, action)
        if not ts:
            print(f"  No transitions for ({state}, {action})")
            return
        rows = []
        for p, ns, r in ts:
            rows.append({"P(s'|s,a)": f"{p:.2f}", "Next State (s')": ns, "Reward R": r})
        df = pd.DataFrame(rows)
        print(f"\n  State: {state} | Action: {action}")
        print(f"  {'─'*40}")
        display(df)

print("✅ MDP base class ready!")

---
## 🌍 Environment 1: 5×5 GridWorld

A classic grid where an agent moves in 4 directions. Walls block movement, there's a goal (+10) and a pit (-10).

```
🤖  ·   ·   ·   ·
 ·  🧱   ·  🧱   ·
 ·   ·   ·   ·   ·
 ·  🧱   ·   ·  🕳️
 ·   ·   ·   ·  🏆
```

In [ ]:
class GridWorld(MDP):
    """5x5 GridWorld with walls, goal, and pit."""

    ACTION_NAMES = {0: '←', 1: '↓', 2: '→', 3: '↑'}
    DELTAS = {0: (0,-1), 1: (1,0), 2: (0,1), 3: (-1,0)}

    def __init__(self, gamma=0.9, slip=0.0):
        self.size = 5
        self.walls = {(1,1), (1,3), (3,1)}
        self.goal = (4,4)
        self.pit = (3,4)
        self.start = (0,0)
        self.slip = slip

        # Build MDP components
        states = [(r,c) for r in range(5) for c in range(5) if (r,c) not in self.walls]
        actions_map = {}
        transitions = {}
        rewards = {}

        for s in states:
            if s == self.goal or s == self.pit:
                actions_map[s] = []
                continue
            actions_map[s] = [0, 1, 2, 3]
            for a in range(4):
                trans_list = []
                for actual_a in range(4):
                    if slip == 0:
                        if actual_a != a:
                            continue
                        prob = 1.0
                    else:
                        prob = (1 - slip + slip/4) if actual_a == a else slip/4
                    if prob < 1e-9:
                        continue

                    dr, dc = self.DELTAS[actual_a]
                    nr, nc = s[0]+dr, s[1]+dc
                    if nr<0 or nr>=5 or nc<0 or nc>=5 or (nr,nc) in self.walls:
                        nr, nc = s
                    ns = (nr, nc)

                    # Merge duplicate next states
                    found = False
                    for i, (p, existing_ns) in enumerate(trans_list):
                        if existing_ns == ns:
                            trans_list[i] = (p + prob, ns)
                            found = True
                            break
                    if not found:
                        trans_list.append((prob, ns))

                    # Rewards
                    if ns == self.goal:
                        rewards[(s, a, ns)] = 10.0
                    elif ns == self.pit:
                        rewards[(s, a, ns)] = -10.0
                    else:
                        rewards[(s, a, ns)] = -0.1

                transitions[(s, a)] = trans_list

        super().__init__(states, actions_map, transitions, rewards, gamma)

    def visualize(self, values=None, policy=None, title="GridWorld 5×5"):
        """Visualize the grid with optional value heatmap and policy arrows."""
        fig, ax = plt.subplots(1, 1, figsize=(7, 7))
        ax.set_xlim(-0.5, 4.5)
        ax.set_ylim(-0.5, 4.5)
        ax.set_aspect('equal')
        ax.invert_yaxis()

        arrow_map = {0: (0, -0.3), 1: (0.3, 0), 2: (0, 0.3), 3: (-0.3, 0)}

        for r in range(5):
            for c in range(5):
                if (r,c) in self.walls:
                    ax.add_patch(plt.Rectangle((c-0.5, r-0.5), 1, 1, color='#334155'))
                    ax.text(c, r, '🧱', ha='center', va='center', fontsize=20)
                elif (r,c) == self.goal:
                    ax.add_patch(plt.Rectangle((c-0.5, r-0.5), 1, 1, color='#064e3b', alpha=0.5))
                    ax.text(c, r, '🏆', ha='center', va='center', fontsize=20)
                elif (r,c) == self.pit:
                    ax.add_patch(plt.Rectangle((c-0.5, r-0.5), 1, 1, color='#450a0a', alpha=0.5))
                    ax.text(c, r, '🕳️', ha='center', va='center', fontsize=20)
                else:
                    if values is not None and (r,c) in values:
                        v = values[(r,c)]
                        all_v = [values[s] for s in values]
                        mn, mx = min(all_v), max(all_v)
                        t = (v - mn) / (mx - mn + 1e-8)
                        color = plt.cm.RdYlGn(t)
                        ax.add_patch(plt.Rectangle((c-0.5, r-0.5), 1, 1, color=color, alpha=0.4))
                        ax.text(c, r+0.3, f"{v:.2f}", ha='center', va='center',
                                fontsize=8, color='white', fontweight='bold')
                    else:
                        ax.add_patch(plt.Rectangle((c-0.5, r-0.5), 1, 1, color='#1e293b', alpha=0.3))

                    if (r,c) == self.start:
                        ax.text(c, r-0.15, '🤖', ha='center', va='center', fontsize=16)

                    if policy is not None and (r,c) in policy:
                        a = policy[(r,c)]
                        dy, dx = arrow_map[a]
                        ax.annotate('', xy=(c+dx, r+dy), xytext=(c, r),
                                   arrowprops=dict(arrowstyle='->', color='#22d3ee', lw=2.5))

                # Grid lines
                ax.add_patch(plt.Rectangle((c-0.5, r-0.5), 1, 1, fill=False,
                            edgecolor='#334155', linewidth=0.5))
                ax.text(c+0.35, r-0.35, f"{r},{c}", ha='center', va='center',
                       fontsize=6, color='#475569')

        ax.set_title(title, fontsize=14, fontweight='bold', color='#22d3ee', pad=10)
        ax.set_xticks([])
        ax.set_yticks([])
        for spine in ax.spines.values():
            spine.set_visible(False)
        plt.tight_layout()
        plt.show()


# Create and visualize
gw = GridWorld(gamma=0.9, slip=0.0)
gw.print_summary()
gw.visualize(title="GridWorld 5×5 — Deterministic")

In [ ]:
# Inspect transitions for a specific state and action
print("═" * 50)
print("  DETERMINISTIC (slip=0)")
print("═" * 50)
gw_det = GridWorld(slip=0.0)
gw_det.show_transitions_table((0,0), 2)  # State (0,0), Action → (Right)
gw_det.show_transitions_table((2,3), 1)  # State (2,3), Action ↓ (Down) — near pit!

print("\n" + "═" * 50)
print("  STOCHASTIC (slip=0.2)")
print("═" * 50)
gw_sto = GridWorld(slip=0.2)
gw_sto.show_transitions_table((0,0), 2)  # Same state, same action — but now stochastic!
gw_sto.show_transitions_table((2,3), 1)  # Near pit — see the danger!

### 🔑 Key Insight

Notice the difference:
- **Deterministic**: Action → always goes where intended (P=1.0)
- **Stochastic**: Action → 80% intended, 5% each other direction. Near the pit, this means there's a chance of falling in even with the "right" action!

This is why **stochastic MDPs require more cautious policies**.

---
## 🧊 Environment 2: FrozenLake 4×4

The classic Gymnasium environment. Walk across a frozen lake — but some tiles are holes!

```
S  F  F  F
F  H  F  H
F  F  F  H
H  F  F  G
```
S=Start, F=Frozen, H=Hole, G=Goal

In [ ]:
class FrozenLakeMDP(MDP):
    """FrozenLake 4x4 as an explicit MDP."""

    MAP = [
        ['S','F','F','F'],
        ['F','H','F','H'],
        ['F','F','F','H'],
        ['H','F','F','G']
    ]
    ACTION_NAMES = {0: '←', 1: '↓', 2: '→', 3: '↑'}
    DELTAS = {0: (0,-1), 1: (1,0), 2: (0,1), 3: (-1,0)}

    def __init__(self, gamma=0.95, is_slippery=True):
        self.size = 4
        self.is_slippery = is_slippery
        self.holes = set()
        self.goal = None
        self.start = None

        for r in range(4):
            for c in range(4):
                if self.MAP[r][c] == 'H': self.holes.add((r,c))
                elif self.MAP[r][c] == 'G': self.goal = (r,c)
                elif self.MAP[r][c] == 'S': self.start = (r,c)

        states = [(r,c) for r in range(4) for c in range(4)]
        terminals = self.holes | {self.goal}
        actions_map = {}
        transitions = {}
        rewards = {}

        for s in states:
            if s in terminals:
                actions_map[s] = []
                continue
            actions_map[s] = [0,1,2,3]
            for a in range(4):
                trans_list = []
                if is_slippery:
                    possible = [(a-1)%4, a, (a+1)%4]
                else:
                    possible = [a]

                for actual_a in possible:
                    prob = 1/3 if is_slippery else 1.0
                    dr, dc = self.DELTAS[actual_a]
                    nr, nc = s[0]+dr, s[1]+dc
                    if nr<0 or nr>=4 or nc<0 or nc>=4:
                        nr, nc = s
                    ns = (nr, nc)

                    found = False
                    for i, (p, existing_ns) in enumerate(trans_list):
                        if existing_ns == ns:
                            trans_list[i] = (p + prob, ns)
                            found = True
                            break
                    if not found:
                        trans_list.append((prob, ns))

                    if ns == self.goal:
                        rewards[(s, a, ns)] = 1.0
                    elif ns in self.holes:
                        rewards[(s, a, ns)] = -1.0
                    else:
                        rewards[(s, a, ns)] = -0.01

                transitions[(s, a)] = trans_list

        super().__init__(states, actions_map, transitions, rewards, gamma)

    def visualize(self, values=None, policy=None, title="FrozenLake 4×4"):
        fig, ax = plt.subplots(1, 1, figsize=(6, 6))
        ax.set_xlim(-0.5, 3.5); ax.set_ylim(-0.5, 3.5)
        ax.set_aspect('equal'); ax.invert_yaxis()
        arrow_map = {0: (0,-0.3), 1: (0.3,0), 2: (0,0.3), 3: (-0.3,0)}

        for r in range(4):
            for c in range(4):
                cell = self.MAP[r][c]
                if cell == 'H':
                    ax.add_patch(plt.Rectangle((c-0.5,r-0.5),1,1,color='#450a0a',alpha=0.5))
                    ax.text(c, r, '🕳️', ha='center', va='center', fontsize=18)
                elif cell == 'G':
                    ax.add_patch(plt.Rectangle((c-0.5,r-0.5),1,1,color='#064e3b',alpha=0.5))
                    ax.text(c, r, '🏆', ha='center', va='center', fontsize=18)
                else:
                    if values and (r,c) in values:
                        v = values[(r,c)]
                        all_v = [values[s] for s in values if s not in self.holes]
                        mn, mx = min(all_v), max(all_v)
                        t = (v-mn)/(mx-mn+1e-8)
                        ax.add_patch(plt.Rectangle((c-0.5,r-0.5),1,1,color=plt.cm.RdYlGn(t),alpha=0.4))
                        ax.text(c, r+0.3, f"{v:.3f}", ha='center', va='center', fontsize=7, color='white')
                    else:
                        ax.add_patch(plt.Rectangle((c-0.5,r-0.5),1,1,color='#1e3a5f',alpha=0.3))
                    if cell == 'S':
                        ax.text(c, r-0.1, '🤖', ha='center', va='center', fontsize=14)
                    if policy and (r,c) in policy:
                        dy, dx = arrow_map[policy[(r,c)]]
                        ax.annotate('', xy=(c+dx,r+dy), xytext=(c,r),
                                   arrowprops=dict(arrowstyle='->',color='#22d3ee',lw=2))
                ax.add_patch(plt.Rectangle((c-0.5,r-0.5),1,1,fill=False,edgecolor='#334155',lw=0.5))

        ax.set_title(title, fontsize=13, fontweight='bold', color='#22d3ee', pad=8)
        ax.set_xticks([]); ax.set_yticks([])
        for s in ax.spines.values(): s.set_visible(False)
        plt.tight_layout(); plt.show()


fl = FrozenLakeMDP(gamma=0.95, is_slippery=True)
fl.print_summary()
fl.visualize(title="FrozenLake 4×4 — Slippery")

---
## 🚦 Environment 3: Traffic Light Controller

An agent controls a traffic light at an intersection. States represent traffic density and light phase.

In [ ]:
class TrafficLightMDP(MDP):
    """Traffic light controller MDP.
    States: (traffic_level, light_phase)
      traffic_level: low, medium, high
      light_phase: green_NS, green_EW
    Actions: keep, switch
    """

    def __init__(self, gamma=0.9):
        traffic_levels = ['low', 'medium', 'high']
        phases = ['green_NS', 'green_EW']
        states = [(t, p) for t in traffic_levels for p in phases]

        actions_map = {s: ['keep', 'switch'] for s in states}
        transitions = {}
        rewards = {}

        # Traffic dynamics
        for traffic in traffic_levels:
            for phase in phases:
                s = (traffic, phase)
                for action in ['keep', 'switch']:
                    new_phase = phase if action == 'keep' else \
                               ('green_EW' if phase == 'green_NS' else 'green_NS')
                    trans = []

                    if traffic == 'low':
                        trans = [(0.7, ('low', new_phase)),
                                 (0.3, ('medium', new_phase))]
                        rewards[(s, action, ('low', new_phase))] = 1.0
                        rewards[(s, action, ('medium', new_phase))] = 0.0
                    elif traffic == 'medium':
                        if action == 'switch':
                            trans = [(0.4, ('low', new_phase)),
                                     (0.5, ('medium', new_phase)),
                                     (0.1, ('high', new_phase))]
                        else:
                            trans = [(0.2, ('low', new_phase)),
                                     (0.4, ('medium', new_phase)),
                                     (0.4, ('high', new_phase))]
                        rewards[(s, action, ('low', new_phase))] = 1.0
                        rewards[(s, action, ('medium', new_phase))] = -0.5
                        rewards[(s, action, ('high', new_phase))] = -2.0
                    else:  # high
                        if action == 'switch':
                            trans = [(0.3, ('medium', new_phase)),
                                     (0.5, ('high', new_phase)),
                                     (0.2, ('low', new_phase))]
                        else:
                            trans = [(0.1, ('medium', new_phase)),
                                     (0.9, ('high', new_phase))]
                        rewards[(s, action, ('medium', new_phase))] = -0.5
                        rewards[(s, action, ('high', new_phase))] = -3.0
                        rewards[(s, action, ('low', new_phase))] = 1.0

                    transitions[(s, action)] = trans

        super().__init__(states, actions_map, transitions, rewards, gamma)


traffic = TrafficLightMDP(gamma=0.9)
traffic.print_summary()

# Show key transitions
traffic.show_transitions_table(('high', 'green_NS'), 'keep')
traffic.show_transitions_table(('high', 'green_NS'), 'switch')
print("\n💡 Notice: 'switch' when traffic is high → better chance of reducing congestion!")

---
## 🌡️ Environment 4: Thermostat Controller

An agent controls a room thermostat. States are temperatures, actions are heat/cool/off.

In [ ]:
class ThermostatMDP(MDP):
    """Thermostat controller.
    States: cold (< 18°C), comfortable (18-24°C), hot (> 24°C)
    Actions: heat, cool, off
    Goal: stay in comfortable zone with minimum energy.
    """

    def __init__(self, gamma=0.9):
        states = ['cold', 'comfortable', 'hot']
        actions_map = {s: ['heat', 'cool', 'off'] for s in states}
        transitions = {}
        rewards = {}

        # Cold state
        transitions[('cold','heat')] = [(0.8,'comfortable'), (0.2,'cold')]
        transitions[('cold','cool')] = [(0.95,'cold'), (0.05,'comfortable')]
        transitions[('cold','off')]  = [(0.7,'cold'), (0.3,'comfortable')]

        # Comfortable state
        transitions[('comfortable','heat')] = [(0.6,'comfortable'), (0.4,'hot')]
        transitions[('comfortable','cool')] = [(0.6,'comfortable'), (0.4,'cold')]
        transitions[('comfortable','off')]  = [(0.8,'comfortable'), (0.1,'cold'), (0.1,'hot')]

        # Hot state
        transitions[('hot','heat')] = [(0.2,'comfortable'), (0.8,'hot')]
        transitions[('hot','cool')] = [(0.8,'comfortable'), (0.2,'hot')]
        transitions[('hot','off')]  = [(0.3,'comfortable'), (0.7,'hot')]

        # Rewards: comfortable=+2, energy cost for heat/cool=-0.5, extreme temps=-1
        for s in states:
            for a in ['heat','cool','off']:
                for prob, ns in transitions[(s,a)]:
                    r = 2.0 if ns == 'comfortable' else -1.0
                    if a in ['heat','cool']:
                        r -= 0.5  # energy cost
                    rewards[(s, a, ns)] = r

        super().__init__(states, actions_map, transitions, rewards, gamma)


thermo = ThermostatMDP(gamma=0.9)
thermo.print_summary()

for s in ['cold', 'comfortable', 'hot']:
    for a in ['heat', 'cool', 'off']:
        thermo.show_transitions_table(s, a)

print("\n💡 Key tradeoff: heating when comfortable risks overheating, but 'off' slowly drifts!")

---
## 🎰 Environment 5: Multi-Armed Bandit (Contextual)

A simplified slot machine problem framed as an MDP — the agent picks which machine to play.

In [ ]:
class BanditMDP(MDP):
    """3-Armed Contextual Bandit as MDP.
    States: morning, afternoon, evening (context)
    Actions: machine_A, machine_B, machine_C
    Each machine has different win rates depending on time of day.
    """

    def __init__(self, gamma=0.9):
        states = ['morning', 'afternoon', 'evening']
        actions_map = {s: ['machine_A', 'machine_B', 'machine_C'] for s in states}
        transitions = {}
        rewards = {}

        # Win probabilities: (state, action) -> P(win)
        win_probs = {
            ('morning', 'machine_A'): 0.7, ('morning', 'machine_B'): 0.3, ('morning', 'machine_C'): 0.5,
            ('afternoon','machine_A'): 0.4, ('afternoon','machine_B'): 0.6, ('afternoon','machine_C'): 0.5,
            ('evening',  'machine_A'): 0.2, ('evening',  'machine_B'): 0.5, ('evening',  'machine_C'): 0.8,
        }

        # Time transitions (cyclic)
        next_time = {'morning': 'afternoon', 'afternoon': 'evening', 'evening': 'morning'}

        for s in states:
            ns = next_time[s]
            for a in actions_map[s]:
                pw = win_probs[(s, a)]
                transitions[(s, a)] = [(1.0, ns)]  # always move to next time
                rewards[(s, a, ns)] = pw * 10 - (1-pw) * 2  # expected: win=+10, lose=-2

        super().__init__(states, actions_map, transitions, rewards, gamma)


bandit = BanditMDP(gamma=0.9)
bandit.print_summary()

print("\n🎰 Expected rewards per (state, action):")
print("═" * 55)
rows = []
for s in ['morning', 'afternoon', 'evening']:
    row = {'Time': s}
    for a in ['machine_A', 'machine_B', 'machine_C']:
        ts = bandit.get_transitions(s, a)
        row[a] = f"{ts[0][2]:.1f}"
    rows.append(row)
display(pd.DataFrame(rows).set_index('Time'))
print("\n💡 Optimal: A in morning, B in afternoon, C in evening!")

---
## 🏭 Environment 6: Inventory Management

A store manages inventory. States are stock levels, actions are order quantities.

In [ ]:
class InventoryMDP(MDP):
    """Inventory Management.
    States: stock level (0, 1, 2, 3, 4)
    Actions: order_0, order_1, order_2
    Demand: stochastic (0, 1, or 2 items sold per period)
    Costs: holding cost + stockout penalty + order cost
    """

    def __init__(self, gamma=0.9):
        max_stock = 4
        states = list(range(max_stock + 1))  # 0 to 4
        actions_map = {s: ['order_0', 'order_1', 'order_2'] for s in states}
        transitions = {}
        rewards = {}

        demand_probs = {0: 0.3, 1: 0.5, 2: 0.2}  # P(demand = d)
        order_cost = 1.0   # per unit ordered
        hold_cost = 0.5    # per unit in stock
        sale_revenue = 3.0 # per unit sold
        stockout_penalty = -4.0  # per unit of unmet demand

        for stock in states:
            for action in ['order_0', 'order_1', 'order_2']:
                order_qty = int(action.split('_')[1])
                after_order = min(stock + order_qty, max_stock)
                o_cost = order_qty * order_cost

                trans = []
                for demand, dp in demand_probs.items():
                    sold = min(demand, after_order)
                    unmet = demand - sold
                    new_stock = after_order - sold

                    reward = sold * sale_revenue - o_cost - new_stock * hold_cost + unmet * stockout_penalty

                    found = False
                    for i, (p, ns) in enumerate(trans):
                        if ns == new_stock:
                            trans[i] = (p + dp, ns)
                            found = True
                            break
                    if not found:
                        trans.append((dp, new_stock))

                    rewards[(stock, action, new_stock)] = reward

                transitions[(stock, action)] = trans

        super().__init__(states, actions_map, transitions, rewards, gamma)


inv = InventoryMDP(gamma=0.9)
inv.print_summary()

# Show what happens at stock=0 (empty!)
for a in ['order_0', 'order_1', 'order_2']:
    inv.show_transitions_table(0, a)
print("\n💡 At stock=0, not ordering → guaranteed stockout penalty!")

---
## 🤖 Environment 7: Robot Navigation (Rooms)

A robot navigates between rooms through doors. Some doors may be locked (stochastic).

In [ ]:
class RobotRoomsMDP(MDP):
    """Robot navigating 4 rooms with doors.
    ┌──────┬──────┐
    │ Rm A │ Rm B │
    │      D      │
    ├──D───┼──D───┤
    │ Rm C │ Rm D │
    │  ⚡  │  🏆  │
    └──────┴──────┘
    Rm C has charging station (small reward)
    Rm D is the goal (big reward)
    Doors may be locked with some probability.
    """

    def __init__(self, gamma=0.9, lock_prob=0.2):
        states = ['room_A', 'room_B', 'room_C', 'room_D']
        # Adjacency: which rooms are connected
        neighbors = {
            'room_A': ['room_B', 'room_C'],
            'room_B': ['room_A', 'room_D'],
            'room_C': ['room_A', 'room_D'],
            'room_D': ['room_B', 'room_C'],
        }

        actions_map = {}
        transitions = {}
        rewards = {}

        for s in states:
            acts = [f'go_{n}' for n in neighbors[s]] + ['stay']
            actions_map[s] = acts

            for a in acts:
                if a == 'stay':
                    transitions[(s, a)] = [(1.0, s)]
                    rewards[(s, a, s)] = -0.1
                    if s == 'room_C':
                        rewards[(s, a, s)] = 1.0  # charging
                else:
                    target = a.replace('go_', '')
                    transitions[(s, a)] = [
                        (1 - lock_prob, target),  # door open
                        (lock_prob, s)            # door locked
                    ]
                    if target == 'room_D':
                        rewards[(s, a, target)] = 10.0  # goal!
                    else:
                        rewards[(s, a, target)] = -0.5  # movement cost
                    rewards[(s, a, s)] = -0.5  # failed attempt

        super().__init__(states, actions_map, transitions, rewards, gamma)


robot = RobotRoomsMDP(gamma=0.9, lock_prob=0.2)
robot.print_summary()

robot.show_transitions_table('room_A', 'go_room_B')
robot.show_transitions_table('room_B', 'go_room_D')
print("\n💡 20% chance any door is locked → agent must plan for failure!")
print("   Should it go A→B→D or A→C→D? Depends on lock probability.")

---
## ⚡ Computing V(s) and Q(s,a)

Now let's compute **value functions** for all our environments using the Bellman equation:

$$V^{\pi}(s) = \sum_a \pi(a|s) \sum_{s'} P(s'|s,a) [R(s,a,s') + \gamma V^{\pi}(s')]$$

$$Q^{\pi}(s,a) = \sum_{s'} P(s'|s,a) [R(s,a,s') + \gamma V^{\pi}(s')]$$

In [ ]:
def policy_evaluation(mdp, policy, theta=1e-8, max_iter=1000):
    """Compute V^pi for a given policy.
    policy: dict {state: action}
    Returns: dict {state: value}
    """
    V = {s: 0.0 for s in mdp.states}

    for iteration in range(max_iter):
        delta = 0
        for s in mdp.states:
            if s not in policy or not mdp.actions.get(s):
                continue
            a = policy[s]
            new_v = 0
            for prob, ns, reward in mdp.get_transitions(s, a):
                new_v += prob * (reward + mdp.gamma * V[ns])
            delta = max(delta, abs(V[s] - new_v))
            V[s] = new_v

        if delta < theta:
            break

    return V


def compute_q_values(mdp, V):
    """Compute Q(s,a) for all state-action pairs."""
    Q = {}
    for s in mdp.states:
        for a in mdp.actions.get(s, []):
            q = 0
            for prob, ns, reward in mdp.get_transitions(s, a):
                q += prob * (reward + mdp.gamma * V[ns])
            Q[(s, a)] = q
    return Q


def policy_improvement(mdp, V):
    """Compute greedy policy from V."""
    policy = {}
    for s in mdp.states:
        if not mdp.actions.get(s):
            continue
        best_a = None
        best_v = -float('inf')
        for a in mdp.actions[s]:
            q = 0
            for prob, ns, reward in mdp.get_transitions(s, a):
                q += prob * (reward + mdp.gamma * V[ns])
            if q > best_v:
                best_v = q
                best_a = a
        policy[s] = best_a
    return policy


def policy_iteration(mdp, max_iter=100):
    """Full policy iteration → returns optimal V* and π*."""
    # Start with arbitrary policy
    policy = {}
    for s in mdp.states:
        if mdp.actions.get(s):
            policy[s] = mdp.actions[s][0]

    for i in range(max_iter):
        V = policy_evaluation(mdp, policy)
        new_policy = policy_improvement(mdp, V)
        if new_policy == policy:
            print(f"  ✅ Policy converged in {i+1} iterations!")
            break
        policy = new_policy

    return V, policy


print("✅ Value function and policy iteration functions ready!")

---
## 📊 Solve All Environments!

Let's find the optimal policy and value function for each environment.

In [ ]:
# === GRIDWORLD ===
print("🌍 GridWorld 5×5 (Deterministic)")
gw_det = GridWorld(gamma=0.9, slip=0.0)
V_gw, pi_gw = policy_iteration(gw_det)
gw_det.visualize(values=V_gw, policy=pi_gw, title="GridWorld — Optimal Policy (Deterministic)")

print("\n🌍 GridWorld 5×5 (Stochastic, slip=0.2)")
gw_sto = GridWorld(gamma=0.9, slip=0.2)
V_gws, pi_gws = policy_iteration(gw_sto)
gw_sto.visualize(values=V_gws, policy=pi_gws, title="GridWorld — Optimal Policy (Stochastic)")

print("\n💡 Compare: stochastic policy avoids cells near the pit!")

In [ ]:
# === FROZENLAKE ===
print("🧊 FrozenLake 4×4")
fl = FrozenLakeMDP(gamma=0.95, is_slippery=True)
V_fl, pi_fl = policy_iteration(fl)
fl.visualize(values=V_fl, policy=pi_fl, title="FrozenLake — Optimal Policy (Slippery)")

# Q-values for start state
Q_fl = compute_q_values(fl, V_fl)
print("\nQ-values at START (0,0):")
for a in range(4):
    q = Q_fl.get(((0,0), a), 0)
    star = " ★ BEST" if q == max(Q_fl.get(((0,0),i),0) for i in range(4)) else ""
    print(f"  {FrozenLakeMDP.ACTION_NAMES[a]} Q = {q:.4f}{star}")

In [ ]:
# === TRAFFIC LIGHT ===
print("🚦 Traffic Light Controller")
traffic = TrafficLightMDP(gamma=0.9)
V_tr, pi_tr = policy_iteration(traffic)

print("\nOptimal Policy:")
for s in sorted(V_tr.keys(), key=str):
    print(f"  {str(s):35s} → {pi_tr.get(s, '—'):10s}  V={V_tr[s]:.2f}")

print("\n💡 When traffic is high → switch! When low → keep current phase.")

In [ ]:
# === THERMOSTAT ===
print("🌡️ Thermostat Controller")
thermo = ThermostatMDP(gamma=0.9)
V_th, pi_th = policy_iteration(thermo)

print("\nOptimal Policy:")
for s in ['cold', 'comfortable', 'hot']:
    print(f"  {s:15s} → {pi_th[s]:8s}  V={V_th[s]:.2f}")

print("\n💡 Cold→heat, Hot→cool, Comfortable→off (save energy!)")

In [ ]:
# === ALL Q-VALUES HEATMAPS ===
print("📊 Q-Value Heatmaps")

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Traffic Q-values
traffic_states = sorted(set(s for (s,a) in compute_q_values(traffic, V_tr).keys()), key=str)
traffic_actions = ['keep', 'switch']
q_matrix = np.zeros((len(traffic_states), len(traffic_actions)))
Q_tr = compute_q_values(traffic, V_tr)
for i, s in enumerate(traffic_states):
    for j, a in enumerate(traffic_actions):
        q_matrix[i,j] = Q_tr.get((s,a), 0)
sns.heatmap(q_matrix, ax=axes[0], annot=True, fmt='.2f', cmap='RdYlGn',
           xticklabels=traffic_actions, yticklabels=[str(s) for s in traffic_states])
axes[0].set_title('Traffic Light Q(s,a)', color='#22d3ee', fontweight='bold')

# Thermostat Q-values
thermo_states = ['cold', 'comfortable', 'hot']
thermo_actions = ['heat', 'cool', 'off']
q_matrix2 = np.zeros((3, 3))
Q_th = compute_q_values(thermo, V_th)
for i, s in enumerate(thermo_states):
    for j, a in enumerate(thermo_actions):
        q_matrix2[i,j] = Q_th.get((s,a), 0)
sns.heatmap(q_matrix2, ax=axes[1], annot=True, fmt='.2f', cmap='RdYlGn',
           xticklabels=thermo_actions, yticklabels=thermo_states)
axes[1].set_title('Thermostat Q(s,a)', color='#22d3ee', fontweight='bold')

# Bandit Q-values
bandit = BanditMDP(gamma=0.9)
V_b, pi_b = policy_iteration(bandit)
Q_b = compute_q_values(bandit, V_b)
b_states = ['morning', 'afternoon', 'evening']
b_actions = ['machine_A', 'machine_B', 'machine_C']
q_matrix3 = np.zeros((3, 3))
for i, s in enumerate(b_states):
    for j, a in enumerate(b_actions):
        q_matrix3[i,j] = Q_b.get((s,a), 0)
sns.heatmap(q_matrix3, ax=axes[2], annot=True, fmt='.2f', cmap='RdYlGn',
           xticklabels=['A','B','C'], yticklabels=b_states)
axes[2].set_title('Bandit Q(s,a)', color='#22d3ee', fontweight='bold')

plt.tight_layout()
plt.show()
print("\n💡 Brightest cell in each row = optimal action for that state!")

---
## 🔬 Experiment: Effect of γ (Discount Factor)

How does the discount factor change the optimal policy?

In [ ]:
gammas = [0.1, 0.5, 0.9, 0.99]
fig, axes = plt.subplots(1, 4, figsize=(18, 4.5))

for idx, g in enumerate(gammas):
    gw = GridWorld(gamma=g, slip=0.0)
    V, pi = policy_iteration(gw)

    ax = axes[idx]
    ax.set_xlim(-0.5, 4.5); ax.set_ylim(-0.5, 4.5)
    ax.set_aspect('equal'); ax.invert_yaxis()
    arrow_map = {0:(0,-0.3), 1:(0.3,0), 2:(0,0.3), 3:(-0.3,0)}

    all_v = [V[s] for s in V if s not in gw.walls and not (s==gw.goal or s==gw.pit)]
    mn, mx = min(all_v), max(all_v)

    for r in range(5):
        for c in range(5):
            if (r,c) in gw.walls:
                ax.add_patch(plt.Rectangle((c-0.5,r-0.5),1,1,color='#334155'))
                ax.text(c,r,'🧱',ha='center',va='center',fontsize=14)
            elif (r,c)==gw.goal:
                ax.add_patch(plt.Rectangle((c-0.5,r-0.5),1,1,color='#064e3b',alpha=0.5))
                ax.text(c,r,'🏆',ha='center',va='center',fontsize=14)
            elif (r,c)==gw.pit:
                ax.add_patch(plt.Rectangle((c-0.5,r-0.5),1,1,color='#450a0a',alpha=0.5))
                ax.text(c,r,'🕳️',ha='center',va='center',fontsize=14)
            else:
                v = V.get((r,c), 0)
                t = (v-mn)/(mx-mn+1e-8)
                ax.add_patch(plt.Rectangle((c-0.5,r-0.5),1,1,color=plt.cm.RdYlGn(t),alpha=0.4))
                ax.text(c,r+0.3,f"{v:.1f}",ha='center',va='center',fontsize=7,color='white')
                if (r,c) in pi:
                    a = pi[(r,c)]
                    dy,dx = arrow_map[a]
                    ax.annotate('',xy=(c+dx,r+dy),xytext=(c,r),
                               arrowprops=dict(arrowstyle='->',color='#22d3ee',lw=2))
            ax.add_patch(plt.Rectangle((c-0.5,r-0.5),1,1,fill=False,edgecolor='#334155',lw=0.3))

    ax.set_title(f'γ = {g}', fontsize=12, fontweight='bold', color='#22d3ee')
    ax.set_xticks([]); ax.set_yticks([])
    for s in ax.spines.values(): s.set_visible(False)

plt.suptitle('Effect of Discount Factor γ on Optimal Policy', fontsize=14,
            fontweight='bold', color='white', y=1.02)
plt.tight_layout()
plt.show()

print("💡 γ=0.1: short-sighted — only cells very near the goal have high value")
print("   γ=0.99: far-sighted — the goal's value propagates across the entire grid!")

---
## 🔬 Experiment: Deterministic vs Stochastic

Compare policies side by side.

In [ ]:
print("Deterministic (slip=0) vs Stochastic (slip=0.3)")
print("="*50)

gw_d = GridWorld(gamma=0.9, slip=0.0)
V_d, pi_d = policy_iteration(gw_d)

gw_s = GridWorld(gamma=0.9, slip=0.3)
V_s, pi_s = policy_iteration(gw_s)

gw_d.visualize(values=V_d, policy=pi_d, title="Deterministic (slip=0)")
gw_s.visualize(values=V_s, policy=pi_s, title="Stochastic (slip=0.3)")

# Show differences
print("\nPolicy differences (cells where arrows differ):")
arrow_names = {0: '←', 1: '↓', 2: '→', 3: '↑'}
for s in sorted(pi_d.keys()):
    if s in pi_s and pi_d[s] != pi_s[s]:
        print(f"  State {s}: deterministic={arrow_names[pi_d[s]]}  stochastic={arrow_names[pi_s[s]]}")

print("\n💡 The stochastic policy takes wider detours to avoid accidentally falling in the pit!")

---
## 📝 Summary

### What We Built
| # | Environment | States | Actions | Key Lesson |
|---|------------|--------|---------|------------|
| 1 | GridWorld 5×5 | 22 cells | ←↓→↑ | Spatial navigation, walls, rewards |
| 2 | FrozenLake 4×4 | 16 cells | ←↓→↑ | Slippery surfaces, holes |
| 3 | Traffic Light | 6 states | keep/switch | Real-world control, stochastic demand |
| 4 | Thermostat | 3 states | heat/cool/off | Energy vs comfort tradeoff |
| 5 | Bandit | 3 contexts | 3 machines | Context-dependent rewards |
| 6 | Inventory | 5 levels | order 0/1/2 | Supply chain, stochastic demand |
| 7 | Robot Rooms | 4 rooms | go/stay | Door uncertainty, path planning |

### Key Takeaways
1. **Every environment is an MDP** — states, actions, transitions, rewards, discount
2. **Deterministic vs Stochastic** — stochastic environments need cautious policies
3. **γ controls planning horizon** — low γ = greedy, high γ = far-sighted
4. **Q(s,a)** tells you exactly how good each action is in each state
5. **Policy Iteration** finds the optimal policy for any MDP

---
📘 **Next:** Chapter 3 — Estimating Value Functions with Dynamic Programming  
🔗 [Policy Iteration Interactive Lab](https://mlnjsh.github.io/rl-book-labs/ch3/)